	# Author: Alexander Staub
	## Last changed: 2025.06.29
	## Purpose: the script that cuts the data into pices according to the number of workers employed


In [1]:
#installing packages
import time
import requests
import logging
import pandas as pd
import os
import numpy as np

In [7]:
# --- CONFIGURATION ---

NUM_WORKERS = 3  # The number of parallel scripts you want to run

# ---  Define the path to your sample dataset ---
# MASTER_INPUT_FILE = "//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/chartmetric/chartmetric_ids/chartmetric_ids_sample.csv"

# --- CHANGE: Define the path to your full dataset ---
MASTER_INPUT_FILE = "//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Spotify/1980_2000_songs_artists/final_charted_from_spotify_1980_2000.csv"

# --- Define a new directory where the split input files will be saved ---
WORKER_INPUT_DIR = "//bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/worker_inputs_chartmetric_ids/"

### WHAT THIS CHANGE DOES:
# This separates the logic. This notebook acts as the main setup script.
# It defines a new, dedicated folder to hold the input files for each worker, keeping your project organized.

print(f"Creating worker input directory: {WORKER_INPUT_DIR}")
os.makedirs(WORKER_INPUT_DIR, exist_ok=True)

Creating worker input directory: //bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/worker_inputs_chartmetric_ids/


In [8]:
# --- Load and Prepare the Master Dataset ---
print(f"Loading master dataset from {MASTER_INPUT_FILE}...")
master_df = pd.read_csv(MASTER_INPUT_FILE)

Loading master dataset from //bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Spotify/1980_2000_songs_artists/final_charted_from_spotify_1980_2000.csv...


In [9]:
# Clean the data ONCE before splitting
master_df = master_df.drop_duplicates(subset='spotify_isrc', keep='first').reset_index(drop=True)
# --- IMPORTANT: Initialize the new column for the fresh run ---
master_df["chartmetric_ids"] = np.nan

In [10]:
# --- Split the DataFrame into Chunks for Each Worker ---
id_chunks = np.array_split(master_df, NUM_WORKERS)

c:\Users\User\anaconda3\envs\music_data_chartmetric\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [11]:
# --- Save Each Chunk to its Own File ---
for i, chunk in enumerate(id_chunks):
    part_number = i + 1
    # --- The output path is now dynamic for each worker part. ---
    output_path = os.path.join(WORKER_INPUT_DIR, f"ids_part_{part_number}.csv")
    
    ### WHAT THIS CHANGE DOES:
    # It creates separate, numbered input files (e.g., ids_part_1.csv, ids_part_2.csv).
    # Each file contains a unique and non-overlapping subset of the original IDs.
    
    chunk.to_csv(output_path, index=False)
    print(f"Saved chunk {part_number} with {len(chunk)} IDs to {output_path}")

print("\nController script finished. You can now run the worker notebooks.")

Saved chunk 1 with 8019 IDs to //bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/worker_inputs_chartmetric_ids/ids_part_1.csv
Saved chunk 2 with 8019 IDs to //bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/worker_inputs_chartmetric_ids/ids_part_2.csv
Saved chunk 3 with 8018 IDs to //bigdata.wu.ac.at/delpero/Data_alexander/data/incidental/chartmetric/worker_inputs_chartmetric_ids/ids_part_3.csv

Controller script finished. You can now run the worker notebooks.
